In [1]:
import numpy as np 
import numpy.linalg as LA
import scipy.stats as stat
from langevin_smc import TimeStep, LangevinSMC

In [2]:
from scipy.special import betainc

# Gaussian Case: two distributions with different means and same covariance

## 1. No models 

In [3]:
#initiallazing random seed
np.random.seed(1)

In [4]:
#parameters
d = 1000
R = 10 #distance between the means
mu_0,mu_1 = np.zeros(d),np.zeros(d)
mu_1[0]=np.sqrt(R)
#m = np.random.normal(size = d)
#mu_1 = m/np.linalg.norm(m)

#Sigma = stat.wishart.rvs(df= d, scale = np.eye(d))
Sigma = (R/d)

In [5]:
#generating data
n_sample = 1000


In [6]:
a=5.
isinstance(a,float) or isinstance(a,int)

True

In [29]:
import sampling_tools
gene_d = lambda N: Sigma*np.random.normal(size =(N,d)) +mu_1

phi_ = lambda x: x.dot(mu_1)/(LA.norm(x)*LA.norm(mu_1))

h_ = lambda x: float(x[0]>R)
h_0 = lambda x: float(x[0]>1)
h_lab = lambda x: np.array(x[0]>5, dtype=np.float)

w_d = lambda x: sampling_tools.gaussian_density_ratio(x, m_1=mu_1,C_0 = 1/Sigma, C_1=1/Sigma)

y = gene_d(1000)
Phi = (np.apply_along_axis(phi_, 1, y))
W = (np.apply_along_axis(w_d, 1, y))
(W[:,None]*Phi.reshape(-1,1)).mean()

6.219080303309307e-216

In [31]:
def ImportanceSampling(gen, phi, w, N=1000, alpha = 0.95):
    """ Importance sampling estimator 
        Args:
            gen: generator of iid samples X_i under alternative law      [fun]
            
            phi: score function                                   [fun]
            w: weight function, typically a ratio of densities    [fun]
            
            N: number of samples                                  [1x1] (1000)
            
            alpha: level of confidence interval                   [1x1] (0.95)
            verbose: level of verbosity                           [1x1] (1)

        Returns:
            E_est: estimated expectancy
            s_out: a dictionary containing additional output data
            -s_out['Var_est']: estimated variance
            -s_out['CI_est']: estimated confidence of interval
            
    """
    q = -stat.norm.ppf((1-alpha)/2)
    d = gen(1).shape[-1]
    Y = gen(N) #array of dim (N,d)
    W = np.apply_along_axis(w, 1, Y) #weighting vectors
    Phi = np.apply_along_axis(phi, 1, Y)
    if Phi.ndim<=1:
        Phi = Phi.reshape(-1,1)      #forces Phi to be two dimensional for compatibility with vectorial phi functions
    
    E_est = (W[:,None]*Phi).mean() 
    Var_est = (((W[:,None]*Phi)**2).mean(0)-E_est**2)/N
    CI_est = E_est*np.array([1,1]) + q*np.sqrt(Var_est)*np.array([-1,1])

    s_out = {"Var_est": Var_est, "CI_est": CI_est}
    return E_est, s_out

    




In [1]:
E_true= stat.norm.sf(5)

NameError: name 'stat' is not defined

In [40]:
E_true= stat.norm.sf(5)
E_est, s_out = ImportanceSampling(gen=gene_d, phi=h_lab, w=w_d, N=400000, alpha = 0.95)
CI_in = (E_true>= s_out["CI_est"][0]) and (E_true< s_out["CI_est"][1])
mape = np.abs(E_true-E_est)/E_true

In [38]:
E_est

0.0

In [27]:
E_true

2.866515718791933e-07

## Linear Model test for Langevin-based SMC 

$X \sim  \mathcal{U}(B_{\mathbb{R}^d}(0,\epsilon))$

In [95]:
d = 15
epsilon = 1
c = 0.5
h = 1-c
e_1 = np.array([1]+[0]*(d-1))
P_target = betainc(0.5*(d+1),0.5,(2*epsilon*h-h**2)/(epsilon**2))
V_batch = lambda X: np.clip(c-X[:,0],a_min=0, a_max = np.inf)
gradV_batch = lambda X: -e_1[None]*(X[:,0]<c)[:,None] 


In [96]:
P_target

0.034599676728248596

In [97]:
def uniform_ball_gen(N,R,d):
    x = np.random.normal(0,1,(N,d))  
    norms = np.linalg.norm(x,axis =1)[:,None]
    r = (R*np.random.uniform(size =(N,1)))**(1.0/d)
    x= r*x/norms
    return x

### Naive Monte Carlo

In [98]:
N_mc = 1000
X = uniform_ball_gen(N_mc,epsilon,d)



In [99]:
P_est_MC = np.mean(X[:,0]>c)
error_MC = np.abs(P_target-P_est_MC)/P_target
del X
print(f"Relative error of naive MC estimator :{error_MC}")

Relative error of naive MC estimator :0.42195991722456816


In [100]:
def project_ball(X, R):
    X_norms = np.linalg.norm(X, axis =1)[:,None]
    X_in_ball = X_norms<=R
    return R*X/X_norms*(~X_in_ball)+(X_in_ball)*X

In [101]:
prjct_epsilon = lambda X: project_ball(X, R=0.6)

In [102]:
def projected_langevin_kernel(X,gradV,delta_t,beta, projection=prjct_epsilon):
    G_noise = np.random.normal(size = X.shape)
    X_new =projection(X-delta_t*gradV(X)+np.sqrt(2*delta_t/beta)*G_noise)
    return X_new


In [103]:
betas = np.linspace(0,1000,100)

In [104]:
uniform_gen_epsilon = lambda N: uniform_ball_gen(N,R=epsilon, d=d)

In [105]:
LA.norm(uniform_gen_epsilon(4),axis=1)

array([0.99148454, 0.97384045, 0.98236057, 0.94251624])

In [106]:
LangevinSMC(uniform_gen_epsilon,  V=V_batch, gradV= gradV_batch, l_kernel = projected_langevin_kernel,betas = betas[1:],alpha = 100)

0.0

In [121]:
gen = uniform_gen_epsilon
V=V_batch
gradV= gradV_batch
l_kernel = projected_langevin_kernel
betas = betas
N=1000
alpha = 100
T=1

In [122]:
X = gen(N) # generate N samples

w= (1/N)*np.ones(N)
v = V(X) # computes their potentials
Count_h = N # Number of calls to function V or it's  gradient
delta_t = alpha*TimeStep(V,X,gradV)

g=1
beta_old = 0

In [124]:
for beta in betas[1:10]:
    G = np.exp(-(beta-beta_old)*v) #computes current value fonction
            
    g*= G.mean()
    #G_tilde = G/G.sum()
    #w = w * G_tilde #updates weights

    for t in range(T):
        X=projected_langevin_kernel(X, gradV, delta_t, beta)

    v = V(X)
    print(f'v min:{v.min()}, ratio adv. {(v<=0).mean()}')
    print(f'Local level proba :{g}')
    beta_old = beta


v min:0.0, ratio adv. 0.018
Local level proba :2626987131094047.0
v min:0.0, ratio adv. 0.982
Local level proba :115064952667246.42
v min:0.0, ratio adv. 0.018
Local level proba :113093669167720.98
v min:0.0, ratio adv. 0.982
Local level proba :5471645875294.479
v min:0.0, ratio adv. 0.018
Local level proba :5375183937811.064
v min:0.0, ratio adv. 0.982
Local level proba :284625809158.4497
v min:0.0, ratio adv. 0.018
Local level proba :279626351966.1945
v min:0.0, ratio adv. 0.982
Local level proba :15089769337.036264
v min:0.0, ratio adv. 0.019
Local level proba :14833381267.16211


In [90]:
G

array([0.07052322, 0.07094231, 0.08181071, 0.07809875, 0.10069065,
       0.05985575, 0.08632759, 0.08861802, 0.08247472, 0.07598067,
       0.0761216 , 0.06498536, 0.11386722, 0.08851727, 0.09720747,
       0.08232568, 0.07118774, 0.08850645, 0.0900826 , 0.07374759,
       0.07420802, 0.07007252, 0.08040534, 0.06152253, 0.06771813,
       0.08854958, 0.07390733, 0.09509416, 0.10297033, 0.07731748,
       0.0892293 , 0.09900778, 0.05725562, 0.0587809 , 0.08284706,
       0.0869259 , 0.07535465, 0.08564974, 0.07698647, 0.07712493,
       0.07146362, 0.06693976, 0.06936882, 0.0924148 , 0.09944258,
       0.07090218, 0.08288347, 0.07862621, 0.04922605, 0.07935632,
       0.08774705, 0.08453352, 0.07644145, 0.06162055, 0.10246571,
       0.09717194, 0.06755741, 0.08534813, 0.08315969, 0.08378221,
       0.08634974, 0.1185489 , 0.0710409 , 0.08668215, 0.10094678,
       0.08106466, 0.10396602, 0.081824  , 0.09271306, 0.08380309,
       0.09684894, 0.10090763, 0.08310908, 0.09092459, 0.09560

In [57]:
v.min()

0.27381338429801116

In [77]:
G

array([0.99999921, 0.9999994 , 0.99999914, 0.99999895, 0.99999885,
       0.99999885, 0.99999936, 0.9999993 , 0.99999951, 0.99999886,
       0.9999994 , 0.99999905, 0.99999935, 0.99999883, 0.99999893,
       0.99999939, 0.99999945, 0.99999926, 0.99999927, 0.99999907,
       0.99999941, 0.999999  , 0.9999991 , 0.99999887, 0.99999933,
       0.9999993 , 0.99999913, 0.99999896, 0.99999918, 0.99999894,
       0.99999937, 0.99999958, 0.99999899, 0.99999903, 0.99999936,
       0.99999941, 0.99999884, 0.99999927, 0.99999889, 0.99999948,
       0.99999907, 0.99999904, 0.99999924, 0.99999934, 0.99999886,
       0.99999905, 0.99999933, 0.99999932, 0.99999897, 0.99999896,
       0.99999937, 0.99999932, 0.99999923, 0.99999902, 0.99999924,
       0.99999926, 0.9999992 , 0.9999991 , 0.99999945, 0.99999863,
       0.9999994 , 0.99999953, 0.99999871, 0.99999931, 0.99999915,
       0.99999914, 0.99999932, 0.99999919, 0.99999938, 0.99999878,
       0.99999961, 0.99999884, 0.99999933, 0.99999898, 0.99999